### Outline

1. Use the yfinance library to fetch historical price data for the specified stocks from 2013-01-01 to 2023-12-31.

2. Create functions to calculate the Double Bollinger Bands and generate buy/sell signals based on the strategy.

3. Design a custom backtesting loop to simulate trading with the generated signals, starting with an initial capital of $10,000 and a minimum transaction size of 1 share per trade.

4. Implement functions to calculate the required performance metrics.

In [56]:
import yfinance as yf
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
from dataclasses import dataclass
from tabulate import tabulate

In [57]:
@dataclass
class TradingMetrics:
    total_return: float
    annual_return: float
    annual_volatility: float
    sharpe_ratio: float
    sortino_ratio: float
    max_drawdown: float

In [58]:
# Fetch data from yfinance
def fetch_data(tickers: List[str], start_date: str, end_date: str) -> Dict[str, pd.DataFrame]:
    """
    Fetch historical data for given tickers within the specified date range.
    
    Args:
        tickers (List[str]): List of stock tickers.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
    
    Returns:
        Dict[str, pd.DataFrame]: Dictionary of DataFrames with historical data for each ticker.
    """
    data = {}
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            df = stock.history(start=start_date, end=end_date)
            if not df.empty:
                data[ticker] = df
            else:
                print(f"No data available for {ticker}")
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    return data

In [59]:
# Calculate double bollinger bands
def calculate_double_bollinger_bands(data: pd.DataFrame, window: int = 20, num_std: Tuple[float, float] = (2, 1)) -> pd.DataFrame:
    """
    Calculate Double Bollinger Bands for the given data.
    
    Args:
        data (pd.DataFrame): DataFrame with 'Close' price column.
        window (int): Rolling window for SMA calculation.
        num_std (Tuple[float, float]): Number of standard deviations for outer and inner bands.
    
    Returns:
        pd.DataFrame: DataFrame with added Bollinger Bands columns.
    """
    df = data.copy()
    df['SMA'] = df['Close'].rolling(window=window).mean()
    df['STD'] = df['Close'].rolling(window=window).std()
    
    df['Upper_BB_2'] = df['SMA'] + (num_std[0] * df['STD'])
    df['Lower_BB_2'] = df['SMA'] - (num_std[0] * df['STD'])
    df['Upper_BB_1'] = df['SMA'] + (num_std[1] * df['STD'])
    df['Lower_BB_1'] = df['SMA'] - (num_std[1] * df['STD'])
    
    return df

![DBB](https://static.wixstatic.com/media/b26bc1_5932ea4b79004095b4d528ee9b00d9a9~mv2.png/v1/fill/w_740,h_361,al_c,q_85,usm_0.66_1.00_0.01,enc_auto/b26bc1_5932ea4b79004095b4d528ee9b00d9a9~mv2.png)

The Double Bollinger Band Strategy contains three distinct trading zones:

- The Buy Zone is between lines A1 and B1

- The Neutral Zone is between lines B1 and B2

- The Sell Zone is between lines B2 and A2

When  the price is within the buy zone, it tells us that the uptrend is strong, and that there is a higher chance that the price will continue upward. Similarly, when the price is in the sell zone, a downtrend will probably continue. 

Note: the above is quoted from https://www.3candlereversal.com/post/kathy-lien-s-double-bollinger-band-strategy. 


In [60]:
# Generate Buy/Sell signals based on momentum double BB strategy
def generate_signals(data: pd.DataFrame) -> pd.DataFrame:
    """
    Generate buy/sell signals based on Double Bollinger Bands.
    
    Args:
        data (pd.DataFrame): DataFrame with Bollinger Bands columns.
    
    Returns:
        pd.DataFrame: DataFrame with added 'Signal' column.
    """
    df = data.copy()
    df['Signal'] = 0  # 0: no signal, 1: buy, -1: sell
    
    # Buy signal: price crosses below Lower BB2
    df.loc[df['Close'] < df['Lower_BB_2'], 'Signal'] = -1
    
    # Sell signal: price crosses above Upper BB2
    df.loc[df['Close'] > df['Upper_BB_2'], 'Signal'] = 1
    
    return df

In [61]:
# Design backtest loop with initial capital 10,000 and min. transaction size of 1 trade per share
def backtest(data: pd.DataFrame, initial_capital: float = 10000) -> pd.DataFrame:
    """
    Perform backtesting on the given data with signals.
    
    Args:
        data (pd.DataFrame): DataFrame with 'Close' price and 'Signal' columns.
        initial_capital (float): Initial capital for the backtest.
    
    Returns:
        pd.DataFrame: DataFrame with backtest results including positions and returns.
    """
    df = data.copy()
    df['Position'] = df['Signal'].shift(1)
    df['Returns'] = df['Close'].pct_change()
    df['Strategy_Returns'] = df['Position'] * df['Returns']
    
    df['Cumulative_Returns'] = (1 + df['Strategy_Returns']).cumprod()
    df['Cumulative_Wealth'] = initial_capital * df['Cumulative_Returns']
    
    return df

In [62]:
# Calculate trading metrics to assess effectiveness
def calculate_metrics(returns: pd.Series) -> TradingMetrics:
    """
    Calculate performance metrics based on strategy returns.
    
    Args:
        returns (pd.Series): Series of strategy returns.
    
    Returns:
        TradingMetrics: Dataclass with calculated performance metrics.
    """
    total_return = (returns + 1).prod() - 1
    annual_return = (1 + total_return) ** (252 / len(returns)) - 1
    annual_volatility = returns.std() * np.sqrt(252)
    
    sharpe_ratio = (annual_return - 0.02) / annual_volatility  # Assuming 2% risk-free rate as target
    
    downside_returns = returns[returns < 0]
    sortino_ratio = (annual_return - 0.02) / (downside_returns.std() * np.sqrt(252))
    
    cumulative_returns = (1 + returns).cumprod()
    max_drawdown = (cumulative_returns.cummax() - cumulative_returns).max()
    
    return TradingMetrics(
        total_return=total_return,
        annual_return=annual_return,
        annual_volatility=annual_volatility,
        sharpe_ratio=sharpe_ratio,
        sortino_ratio=sortino_ratio,
        max_drawdown=max_drawdown
    )

In [63]:
# Execute functions
def main():
    tickers = ['MSFT', 'AAPL', 'NVDA', 'AMZN', 'GOOG', 'META', 'TSLA']
    start_date = '2013-01-01'
    end_date = '2023-12-31'

    # Fetch data
    data = fetch_data(tickers, start_date, end_date)

    results = {}
    for ticker, df in data.items():
        # Calculate indicators
        df = calculate_double_bollinger_bands(df)

        # Generate signals
        df = generate_signals(df)

        # Perform backtesting
        backtest_results = backtest(df)

        # Calculate metrics
        metrics = calculate_metrics(backtest_results['Strategy_Returns'])
        results[ticker] = metrics

    # Convert results to DataFrame
    results_data = {
        ticker: {
            'Total Return': f"{metrics.total_return:.2%}",
            'Annual Return': f"{metrics.annual_return:.2%}",
            'Annual Volatility': f"{metrics.annual_volatility:.2%}",
            'Sharpe Ratio': f"{metrics.sharpe_ratio:.2f}",
            'Sortino Ratio': f"{metrics.sortino_ratio:.2f}",
            'Maximum Drawdown': f"{metrics.max_drawdown:.2%}"
        } for ticker, metrics in results.items()
    }
    results_df = pd.DataFrame.from_dict(results_data, orient='index')

    # Print the table
    print(tabulate(results_df, headers='keys', tablefmt='pretty'))

if __name__ == "__main__":
    main()

+------+--------------+---------------+-------------------+--------------+---------------+------------------+
|      | Total Return | Annual Return | Annual Volatility | Sharpe Ratio | Sortino Ratio | Maximum Drawdown |
+------+--------------+---------------+-------------------+--------------+---------------+------------------+
| MSFT |   -53.13%    |    -6.67%     |      10.84%       |    -0.80     |     -0.29     |      68.30%      |
| AAPL |    14.75%    |     1.26%     |      10.13%       |    -0.07     |     -0.03     |      46.01%      |
| NVDA |   -41.13%    |    -4.71%     |      14.52%       |    -0.46     |     -0.20     |      73.86%      |
| AMZN |   -12.53%    |    -1.21%     |      10.81%       |    -0.30     |     -0.13     |      26.16%      |
| GOOG |   -36.76%    |    -4.09%     |      11.06%       |    -0.55     |     -0.23     |      81.01%      |
| META |   -51.93%    |    -6.45%     |      11.08%       |    -0.76     |     -0.35     |      55.54%      |
| TSLA |  

The results are not great, as seen from the negative Sharpe and Sortino ratios for six out of seven stocks. This indicates that only Tesla stock prices follow a strong momentum trend from 2013 to 2023, while the rest not so much. 

Let's try out the mean reversion double BB strategy instead. We simply flip the buy/sell signals around (buy zone between B2 and A2, sell zone between A1 and B1).

In [64]:
# Generate Buy/Sell signals based on mean-reverting double BB strategy
def generate_signals_mean_reverting(data: pd.DataFrame) -> pd.DataFrame:
    """
    Generate buy/sell signals based on Double Bollinger Bands.
    
    Args:
        data (pd.DataFrame): DataFrame with Bollinger Bands columns.
    
    Returns:
        pd.DataFrame: DataFrame with added 'Signal' column.
    """
    df = data.copy()
    df['Signal'] = 0  # 0: no signal, 1: buy, -1: sell
    
    # Buy signal: price crosses below Lower BB2
    df.loc[df['Close'] < df['Lower_BB_2'], 'Signal'] = 1
    
    # Sell signal: price crosses above Upper BB2
    df.loc[df['Close'] > df['Upper_BB_2'], 'Signal'] = -1
    
    return df

In [65]:
# Execute functions for mean-reverting strategy
def main():
    tickers = ['MSFT', 'AAPL', 'NVDA', 'AMZN', 'GOOG', 'META', 'TSLA']
    start_date = '2013-01-01'
    end_date = '2023-12-31'

    # Fetch data
    data = fetch_data(tickers, start_date, end_date)

    results = {}
    for ticker, df in data.items():
        # Calculate indicators
        df = calculate_double_bollinger_bands(df)

        # Generate signals
        df = generate_signals_mean_reverting(df)

        # Perform backtesting
        backtest_results = backtest(df)

        # Calculate metrics
        metrics = calculate_metrics(backtest_results['Strategy_Returns'])
        results[ticker] = metrics

    # Convert results to DataFrame
    results_data = {
        ticker: {
            'Total Return': f"{metrics.total_return:.2%}",
            'Annual Return': f"{metrics.annual_return:.2%}",
            'Annual Volatility': f"{metrics.annual_volatility:.2%}",
            'Sharpe Ratio': f"{metrics.sharpe_ratio:.2f}",
            'Sortino Ratio': f"{metrics.sortino_ratio:.2f}",
            'Maximum Drawdown': f"{metrics.max_drawdown:.2%}"
        } for ticker, metrics in results.items()
    }
    results_df = pd.DataFrame.from_dict(results_data, orient='index')

    # Print the table
    print(tabulate(results_df, headers='keys', tablefmt='pretty'))

if __name__ == "__main__":
    main()

+------+--------------+---------------+-------------------+--------------+---------------+------------------+
|      | Total Return | Annual Return | Annual Volatility | Sharpe Ratio | Sortino Ratio | Maximum Drawdown |
+------+--------------+---------------+-------------------+--------------+---------------+------------------+
| MSFT |    87.40%    |     5.88%     |      10.84%       |     0.36     |     0.19      |      33.76%      |
| AAPL |   -22.15%    |    -2.25%     |      10.13%       |    -0.42     |     -0.25     |      42.92%      |
| NVDA |    34.68%    |     2.75%     |      14.52%       |     0.05     |     0.03      |      42.22%      |
| AMZN |    0.54%     |     0.05%     |      10.81%       |    -0.18     |     -0.10     |      31.75%      |
| GOOG |    38.19%    |     2.99%     |      11.06%       |     0.09     |     0.04      |      33.16%      |
| META |    81.76%    |     5.59%     |      11.08%       |     0.32     |     0.15      |      28.75%      |
| TSLA |  

The results looks slightly better. As expected, the mean-reversion strategy does not work for Tesla because it follows a strong momentum trend.

For Apple, the momentum strategy is clearly a better fit as it gave a positive total return (14.75%) and less negative Sharpe and Sortino ratios.

For Amazon, the mean-reversion strategy is a slightly better fit since it generated a small total return (0.54%) with less negative Sharpe and Sortino ratios.

However, I would hesitate to apply either strategies using double BB since they are clearly below target return rate (assumed 2%). Investing in a cash portfolio or a mid-/long-term bond would likely yield better returns.

## Conclusion

I looked at price data from 2013 to 2023 for the Magnificent 7 on Yahoo Finance. I backtested a simple momentum and mean-reversion strategy using double Bolinger bands. Backtests were done with initial capital of 10,000 and a minimum of 1 trade per share.

An overview of the trading metrics shows that TSLA works well with a momentum strategy while MSFT, NVDA, GOOG and META works well with a mean-reversion strategy. Both strategies do not work well for AAPL and AMZN.